In [1]:
WINDOW_LENGTHS = [10]
DO_BLUR = False
BLACK_THRESH = 3  # x <= threshhold
WHITE_THRESH = 253 # x > thresh

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

In [3]:
sys.path.append(os.path.abspath('..'))
import live_3d_rendering as l3d

In [4]:
import glob
import itertools as it
from typing import Iterator, Optional

celiacs_dir = "images/mar29-higher-res/celiac/"
controls_dir = "images/mar29-higher-res/control/"
img_file_extensions = ['jpg', 'jpeg', 'png', 'JPG', 'JPEG', 'PNG']

def file_iterator(directory, file_ext) -> Iterator[str]:
    return glob.glob(os.path.join(directory, f"*.{file_ext}"), recursive=False)

celiac_files = it.chain(*[file_iterator(celiacs_dir, ext) for ext in img_file_extensions])
control_files = it.chain(*[file_iterator(controls_dir, ext) for ext in img_file_extensions])

In [5]:
from PIL import Image, ImageFilter

if DO_BLUR:
    celiac_ims = [l3d.preprocess(Image.open(f)).filter(ImageFilter.GaussianBlur(5)) for f in celiac_files]
    control_ims = [l3d.preprocess(Image.open(f)).filter(ImageFilter.GaussianBlur(5)) for f in control_files]
else:
    celiac_ims = [l3d.preprocess(Image.open(f), white_thresh=WHITE_THRESH) for f in celiac_files]
    control_ims = [l3d.preprocess(Image.open(f), white_thresh=WHITE_THRESH) for f in control_files]
    

In [6]:
from live_3d_rendering import local_stats as stats

def window_ok(w: stats.Window, black_thresh: int) -> bool:
    """Return True if w is nonempty and at most 1% pixels <= black_thresh."""
    return w.access().size > 0 and np.sum(w.access() <= black_thresh) < 0.01 * w.access().size

In [7]:
# def window_iterator(im: Image.Image, side_len: int, black_thresh: Optional[int] = 10) -> Iterator[stats.Window]:
#     arr = np.array(im)
#     yield stats.Window(arr, 0, 0, *arr.shape)

def window_iterator(im: Image.Image, side_len: int, black_thresh: Optional[int] = 10) -> Iterator[stats.Window]:
    arr = np.array(im)
    for row in range(0, max(1, im.size[0] - side_len), side_len):
        for col in range(0, max(1, im.size[1] - side_len), side_len):
            window = stats.Window(arr, row, col, side_len, side_len)
            if window_ok(window, black_thresh):
                yield window

In [8]:
from live_3d_rendering.stats_functions import (
    local_min,
    local_max,
    local_mean,
    local_variance,
)
import pandas as pd

funcs = [local_min, local_max, local_mean, local_variance, lambda x: np.sqrt(x.var())]
func_names = ['min', 'max', 'mean', 'variance', 'std']

In [9]:
celiac_info_by_wlen = {wlen: [] for wlen in WINDOW_LENGTHS}
control_info_by_wlen = {wlen: [] for wlen in WINDOW_LENGTHS}

for window_len in WINDOW_LENGTHS:
    
    for im in celiac_ims:
        for window in window_iterator(im, window_len, BLACK_THRESH):
            celiac_info_by_wlen[window_len].append(window.get_stats(funcs))
    
    for im in control_ims:
        for window in window_iterator(im, window_len, BLACK_THRESH):
            control_info_by_wlen[window_len].append(window.get_stats(funcs))

In [10]:
for wlen in WINDOW_LENGTHS:
    celiac_df = pd.DataFrame(celiac_info_by_wlen[wlen], columns=func_names)
    celiac_df.to_csv(f"celiac_{wlen}_by_{wlen}.csv", index=False)
    control_df = pd.DataFrame(control_info_by_wlen[wlen], columns=func_names)
    control_df.to_csv(f"control_{wlen}_by_{wlen}.csv", index=False)

# Visualization

In [11]:
def stats_of_wlen(WL):
    celiac = pd.read_csv(f"celiac_{WL}_by_{WL}.csv")
    celiac_means = celiac.mean()
    control = pd.read_csv(f"control_{WL}_by_{WL}.csv")
    control_means = control.mean()
    return pd.DataFrame(
        [celiac_means, control_means],
        columns=['min', 'max', 'mean', 'variance', 'std'],
        index=["celiac", "control"],
    )
from IPython.display import display
print('---------------------------------------------------\n')
for wlen in WINDOW_LENGTHS:
    print(f"{wlen}x{wlen} windows")
    display(stats_of_wlen(wlen))
    print('---------------------------------------------------\n')

---------------------------------------------------

10x10 windows


,min,max,mean,variance,std
celiac,76.199808,91.319558,83.928114,27.844894,3.801672
control,87.653408,100.185581,94.124049,18.938353,3.123945


---------------------------------------------------



### (ignore the rest of this document, it's scratch work)